<a href="https://colab.research.google.com/github/luisxv95-1988/Challenge-de-Clasificaci-n-Biom-dica-con-IA/blob/main/Clasificador_Medico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge de Clasificación Biomédica con IA

Crea una solución de IA que ayude a clasificar investigación médica. Perfecto para desarrollar habilidades avanzadas en Data Science y destacar en tu portafolio profesional.


# 1. Reconocimiento de datos y EDA.

Iniciamos con el reconocimiento de los datos para aprendizaje del modelo a implementar.

Verificaremos la cantidad de datos, la integralidad de la información, los grupos.

Usaremos la biblioteca Pandas para la manipulación de los datos con Python.

Siempre que se ejecute el codigo debemos cargar el archivo de entrenamiento que vamos a utilizar. En este caso 'challenge_data-18-ago.csv'.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

uploaded = files.upload()

# Obtiene el nombre del archivo cargado
file_name1 = next(iter(uploaded))

# Asegúrate de que la ruta del archivo sea la correcta en tu Google Drive
df = pd.read_csv(file_name1, delimiter=';')

# Mostramos las primeras filas para verificar que se cargó correctamente
print("Primeras 3 filas del DataFrame:")
print(df.head(3))

# Verificamos la información general del DataFrame
print("\nInformación del DataFrame:")
df.info()

# Verificamos si hay valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Contar la frecuencia de cada grupo
all_groups = df['group'].str.split('|').explode()
group_counts = all_groups.value_counts()

print("\nFrecuencia de cada grupo:")
print(group_counts)


In [ ]:
# Visualización de la distribución de los grupos
plt.figure(figsize=(12, 8))
ax = sns.barplot(x=group_counts.index, y=group_counts.values, palette='viridis')
plt.title('Distribución de los Grupos Médicos')
plt.xlabel('Dominio Médico')
plt.ylabel('Número de Artículos')
plt.xticks(rotation=45, ha='right')

# Añadir etiquetas de datos
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(),
            f'{int(p.get_height())}',
            fontsize=12, ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Verificamos si hay artículos sin grupo asignado
empty_group_count = df['group'].isnull().sum()

print(f"\nNúmero de filas sin grupo asignado: {empty_group_count}")

# 1.1 Conclusiones EDA:

Análisis de la distribución de etiquetas nos dio la frecuencia de cada grupo.

Vamos a analizar esos resultados y a crear una representación visual para entender mejor el problema.

Resultados del df.info():

El dataset tiene 3565 filas.

No hay valores nulos en ninguna de las columnas (title, abstract, group), lo que es excelente y simplifica el preprocesamiento.

Todas las columnas son de tipo object (texto), como se esperaba.

Resultados de all_groups.value_counts(): Aquí están las frecuencias de cada dominio médico.
Frecuencia de cada grupo:
neurological      1785
cardiovascular    1268
hepatorenal       1091
oncological        601

Reflexiones:

Desequilibrio de Clases: Hay un claro desequilibrio. cardiovascular y neurologico son las clases más frecuentes, mientras que oncologico es la menos frecuente con 601 lineas. Este desequilibrio podría afectar el rendimiento del modelo en las clases minoritarias.

Clasificación Multi-etiqueta: El hecho de que un artículo pueda tener múltiples etiquetas (| en la columna group) significa que no podemos usar un clasificador simple de una sola clase. Tendremos que transformar esta columna en un formato adecuado para el entrenamiento. La técnica más común es la binarización de etiquetas que es la que aplicaremos a continuación.

# 2. Preprocesamiento de Datos y Binarización de Etiquetas.

Antes de alimentar los datos a un modelo de machine learning, necesitamos prepararlos. Esto incluye la binarización de la columna group y la preparación del texto.

Aqui importamos la biblitecas de Numply y sklearn.

**Combinación de Texto:** Para simplificar, unimos el title y el abstract en una sola columna llamada text. Esto asegura que toda la información textual esté disponible para el modelo en una sola "característica".

**Binarización:** La clase MultiLabelBinarizer de scikit-learn es perfecta para este trabajo.

**df['group'].apply(lambda x: x.split('|'))** divide la cadena de grupos en una lista de etiquetas.

**mlb.fit_transform()** crea una matriz donde cada fila corresponde a un artículo y cada columna a un dominio médico. Un 1 indica que el artículo pertenece a ese dominio, y un 0 indica que no. Esta matriz y será nuestra variable objetivo para el entrenamiento.

In [ ]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

# Unimos las columnas 'title' y 'abstract' en una sola
df['text'] = df['title'] + ' ' + df['abstract']

# Binarizamos la columna 'group'
# Primero, separamos las etiquetas
df['groups_list'] = df['group'].apply(lambda x: x.split('|'))

# Creamos una instancia de MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Ajustamos y transformamos los datos para crear la matriz de etiquetas
y = mlb.fit_transform(df['groups_list'])
y_df = pd.DataFrame(y, columns=mlb.classes_)

# Mostramos las primeras 3 filas de la matriz de etiquetas
print("\nPrimeras 3 filas de la matriz de etiquetas binarizadas:")
print(y_df.head(3))
print("\nClases detectadas:")
print(list(mlb.classes_))

# 3. Vectorización del Texto con TF-IDF.

TF-IDF no solo cuenta la frecuencia de una palabra en un documento (Term Frequency), sino que también penaliza las palabras que son muy comunes en todo el corpus (Inverse Document Frequency), como "el", "la", "un" ó (como "the", "a", "is" en Ingles como este caso). Esto significa que las palabras que son únicas o más relevantes para un dominio médico específico (por ejemplo, "cardiaco" en un texto de cardiología) tendrán un peso mayor en la representación vectorial, lo que hace que los datos sean más significativos para el modelo.

Implementación en Python Usaremos TfidfVectorizer de scikit-learn para este paso. También es fundamental dividir los datos en conjuntos de entrenamiento y prueba para poder evaluar el rendimiento del modelo en datos que nunca ha visto.

**Cargamos las bibliotecas siguientes bibliotecas de sklearn**

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

Aqui parametrizamos el set de datos de la siguiente manera: 80% para entrenamiento y 20% para pruebas. (test_size: 0.2)

Se asigna random_state=40 para garantizar la repetibilidad de las pruebas.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Dividimos los datos en conjuntos de entrenamiento y prueba
X = df['text']  # Características (texto)
y = y_df        # Etiquetas binarizadas

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

print("Dimensiones de los conjuntos de datos:")
print(f"Conjunto de entrenamiento (X_train): {X_train.shape}")
print(f"Conjunto de prueba (X_test): {X_test.shape}")
print(f"Etiquetas de entrenamiento (y_train): {y_train.shape}")
print(f"Etiquetas de prueba (y_test): {y_test.shape}")

# Creamos una instancia de TfidfVectorizer
# Podríamos ajustar el max_df, min_df o ngram_range para una mayor optimización
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Ajustamos el vectorizador solo en el conjunto de entrenamiento y lo transformamos
X_train_vec = vectorizer.fit_transform(X_train)

# Usamos el mismo vectorizador para transformar el conjunto de prueba
X_test_vec = vectorizer.transform(X_test)

print("\nDimensiones de los datos vectorizados:")
print(f"Matriz de entrenamiento TF-IDF: {X_train_vec.shape}")
print(f"Matriz de prueba TF-IDF: {X_test_vec.shape}")

# 4. Entrenamiento del Modelo

Ahora que nuestros datos están vectorizados y listos, vamos a entrenar el modelo en el conjunto de entrenamiento (X_train_vec, y_train) y luego a evaluarlo en el conjunto de prueba (X_test_vec, y_test).

El OneVsRestClassifier funciona creando un clasificador binario para cada una de las clases. Por ejemplo, un clasificador para cardiovascular (que distingue entre cardiovascular y "no cardiovascular"), otro para oncological, y así sucesivamente.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import numpy as np

# Creamos una instancia de OneVsRestClassifier con un modelo base de LogisticRegression
# Usamos un solver 'lbfgs' que es eficiente y el C (inverso de la regularización) para controlar el sobreajuste.

model = OneVsRestClassifier(LogisticRegression(solver='lbfgs', C=1.1))

# Entrenamos el modelo con los datos de entrenamiento
print("Entrenando el modelo...")
model.fit(X_train_vec, y_train)
print("¡Entrenamiento completado!")

# Hacemos las predicciones en el conjunto de prueba
y_pred = model.predict(X_test_vec)


## 4.1 Entrenar otros dos modelos de clasificación

Entrenar otro modelo de clasificación adecuado para problemas multi-etiqueta y entrénalo con los datos vectorizados.

En este caso se entrenaron los modelos 'LinearSVC' y 'DecisionTreeClassifier' con el objetivo de evaluar las métricas de F1-Score y elegir el modelo mas ajustado en predicciones.


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier


# Create an instance of OneVsRestClassifier with DecisionTreeClassifier as the base estimator
# We'll use a random_state for reproducibility
model_tree = OneVsRestClassifier(DecisionTreeClassifier(random_state=40))

# Train the new model with the vectorized training data
print("Entrenando el modelo DecisionTreeClassifier...")
# Assuming X_train_vec and y_train are available from previous steps (TF-IDF vectorization)
model_tree.fit(X_train_vec, y_train)
print("¡Entrenamiento de DecisionTreeClassifier completado!")

# Create an instance of OneVsRestClassifier with LinearSVC as the base estimator
model_lsvc = OneVsRestClassifier(LinearSVC(C=3.2, random_state=40, dual=True))
# Train the new model with the vectorized training data
print("Entrenando el modelo LinearSVC...")
model_lsvc.fit(X_train_vec, y_train)
print("¡Entrenamiento de LinearSVC completado!")

## 5. Evaluamos los Modelos Preentrenados

### Metódo

Evaluar los modelos (el actual de Regresión Logística, LinearSVC y Arból de desición) calculando el F1-score ponderado en el conjunto de prueba.

En este caso solo continuaremos con el modelo que tenga el mayor F1-Score.

In [ ]:
# Calculate weighted F1-score for Logistic Regression model
f1_weighted_lr = f1_score(y_test, y_pred, average='weighted')

# Make predictions using the LinearSVC model
y_pred_lsvc = model_lsvc.predict(X_test_vec)

# Calculate weighted F1-score for LinearSVC model
f1_weighted_lsvc = f1_score(y_test, y_pred_lsvc, average='weighted')

# Make predictions using the DecisionTreeClassifier model
y_pred_tree = model_tree.predict(X_test_vec)

# Calculate weighted F1-score for DecisionTreeClassifier model
# Assuming y_test is available from previous steps (train-test split)
f1_weighted_tree = f1_score(y_test, y_pred_tree, average='weighted')

print(f"F1-score ponderado (Regresión Logística): {f1_weighted_lr:.4f}")
print(f"F1-score ponderado (LinearSVC): {f1_weighted_lsvc:.4f}")
print(f"F1-score ponderado (Arból de Desición): {f1_weighted_tree:.4f}")

best_f1 = max(f1_weighted_lr, f1_weighted_lsvc, f1_weighted_tree)

if best_f1 == f1_weighted_lr:
    best_model_name = "Regresión Logística"
elif best_f1 == f1_weighted_lsvc:
    best_model_name = "LinearSVC"
else:
    best_model_name = "Arból de Desición"

print(f"\nEl modelo con el mejor F1-score ponderado es: {best_model_name} ({best_f1:.4f})")

# 6. Evaluación de Métricas

Teniendo en cuenta los resultados del F1-score se selecciona el modelo de LinearSVC para el desarrollo del modelo final.

F1-score ponderado (Regresión Logística): 0.8228 F1-score ponderado (LinearSVC): 0.8876 F1-score ponderado (Árbol de Decisión): 0.8476

El modelo con el mejor F1-score ponderado es: LinearSVC (0.8876)

Se ejecutan todas las métricas para el modelo seleccionado LinearSVR.

El reto pide dos métricas clave: el F1-score ponderado y la matriz de confusión.

El F1-score ponderado es la métrica principal. Combina la precisión (precision) y la exhaustividad (recall) en una sola métrica y la "pondera" (promedia) según el número de ejemplos de cada clase. Esto es crucial en nuestro caso debido al desequilibrio de clases. Un F1-score alto indica un buen balance entre ambas métricas.

La matriz de confusión nos dará una visión detallada del rendimiento del modelo para cada clase, mostrando cuántas predicciones fueron correctas (True Positives, True Negatives) e incorrectas (False Positives, False Negatives).

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Generate the classification report for the LinearSVC model
print("Informe de Clasificación para el Modelo LinearSVC:")
# Ensure target_names are correctly aligned with the columns of y_test
print(classification_report(y_test, y_pred_lsvc, target_names=mlb.classes_))

# Generate and display the confusion matrix for each class
print("\nMatrices de Confusión para cada Clase (LinearSVC):")

# Get the class names from the MultiLabelBinarizer
class_names = mlb.classes_

# Iterate through each class and calculate/display the confusion matrix
for i, class_name in enumerate(class_names):
    print(f"\n--- Matriz de Confusión para '{class_name}' ---")
# Obtener la matriz de confusión para la clase ''{class_name}''
    class_index = list(y_df.columns).index(class_name)
    cm = confusion_matrix(y_test.iloc[:, class_index], y_pred_lsvc[:, class_index])
    # Visualizar la matriz de confusión
    plt.figure(figsize=(2, 2))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No ', 'Si'], yticklabels=['No ', 'Si '])
    plt.title(f'Matriz de Confusión para "{class_name}"')
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.show()



    # Get the true labels and predicted labels for the current class
  #  y_true_class = y_test.iloc[:, i]
  #  y_pred_class = y_pred_lsvc[:, i]

    # Calculate the confusion matrix
    # The matrix will be:
    # [[True Negatives (TN), False Positives (FP)]
    #  [False Negatives (FN), True Positives (TP)]]
   # cm = confusion_matrix(y_true_class, y_pred_class)

 #   print(cm)

    # You can also print TN, FP, FN, TP for clarity
  #  tn, fp, fn, tp = cm.ravel()
 #   print(f"TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}")

# 7. Cargue de Archivo Prueba

En este código el usuario puede cargar los datos etiquetados o no para realizar su respectiva clasificación automática de la investigación médica, cuando los datos están etiquetados el código genera las estadísticas F1-score y Matriz de confusión.

El usuario debe iniciar la ejecución del código y el asistente de datos le solicitara adjuntar el archivo.

In [ ]:
from google.colab import files
import sys

# Prompt the user to upload a file
uploaded = files.upload()

# Get the name of the uploaded file
file_name = next(iter(uploaded))

# Read the uploaded CSV file into a pandas DataFrame
df_new = pd.read_csv(file_name, delimiter=';')

# Display the first few rows of the new DataFrame to verify
print("Primeras 3 filas del nuevo DataFrame:")
print(df_new.head(3))

# Verify the general information of the new DataFrame
print("\nInformación del nuevo DataFrame:")
df_new.info()

# Combine the 'title' and 'abstract' columns into a new 'text' column
df_new['text'] = df_new['title'] + ' ' + df_new['abstract']

# Display the first few rows of the new DataFrame with the added 'text' column
print("Primeras 3 filas del nuevo DataFrame con la columna 'text':")
print(df_new.head(3))

# Transform the 'text' column of the new DataFrame using the fitted vectorizer
X_new_vec = vectorizer.transform(df_new['text'])

# Print the dimensions of the new TF-IDF matrix
print("\nDimensiones de la nueva matriz TF-IDF:")
print(X_new_vec.shape)

# Make predictions using the trained LinearSVC model (model_lsvc)
y_new_pred = model_lsvc.predict(X_new_vec)

# Print the shape of the predictions
print("\nDimensiones de las predicciones para el nuevo DataFrame:")
print(y_new_pred.shape)

# Convert the predicted labels (binary matrix) back to class names
predicted_groups = mlb.inverse_transform(y_new_pred)

# Join the predicted groups (lists of strings) into a single string separated by '|'
predicted_groups_str = ['|'.join(groups) for groups in predicted_groups]

# Add the predicted groups as a new column 'group_predicted' to the original new DataFrame
df_new['group_predicted'] = predicted_groups_str

# Display the first few rows of the DataFrame with the new 'group_predicted' column
print("\nPrimeras 3 filas del DataFrame con la columna 'group_predicted':")
print(df_new.head(3))


# 7.1 Verifica datos de Clasificación

Verificar si el DataFrame original contiene una columna 'group' para determinar si se pueden calcular métricas de evaluación, luego imprimir un mensaje indicando el resultado.

Si no se ejecuta no se muestran los resultados de las pruebas de clasificación por F1-Score y por Matriz de Confusión para cada grupo.



In [ ]:
# Check if the original 'group' column exists in the new DataFrame
if 'group' in df_new.columns:
    print("\nEl archivo cargado contiene la columna 'group'. Procederemos a calcular métricas de evaluación.")
    calculate_metrics = True
else:
    print("\nEl archivo cargado NO contiene la columna 'group'. No se calcularán métricas de evaluación.")
    calculate_metrics = False


# If the 'group' column exists, calculate and display evaluation metrics
if calculate_metrics:
    # Create the 'groups_list' column in the new DataFrame by splitting the 'group' column
    df_new['groups_list'] = df_new['group'].apply(lambda x: x.split('|'))

    # Binarize the actual 'group' column from the new DataFrame
    # Ensure mlb is fitted with all possible classes from the original training data
    y_new_actual = mlb.transform(df_new['groups_list'])

    # Generate the classification report for the LinearSVC model
    print("\nInforme de Clasificación para el Modelo LinearSVC en el nuevo archivo:")
    print(classification_report(y_new_actual, y_new_pred, target_names=mlb.classes_))

    # Generate and display the confusion matrix for each class
    print("\nMatrices de Confusión para cada Clase (LinearSVC) en el nuevo archivo:")

    # Get the class names from the MultiLabelBinarizer
    class_names = mlb.classes_

    # Iterate through each class and calculate/display the confusion matrix
    for i, class_name in enumerate(class_names):
        print(f"\n--- Matriz de Confusión para '{class_name}' ---")

        # Get the true labels and predicted labels for the current class
        y_true_class = y_new_actual[:, i]
        y_pred_class = y_new_pred[:, i]

        # Calculate the confusion matrix
        cm = confusion_matrix(y_true_class, y_pred_class)

        # Visualizar la matriz de confusión
        plt.figure(figsize=(2, 2))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No ', 'Si'], yticklabels=['No ', 'Si '])
        plt.title(f'Matriz de Confusión para "{class_name}"')
        plt.xlabel('Predicho')
        plt.ylabel('Real')
        plt.show()

# 7.2 Mostrar y Guardar los Resultados.

Mostrar las primeras 10 filas del DataFrame con las predicciones y guardar el DataFrame actualizado en un archivo nuevo llamado 'predictions_values.CSV' el cual sera descargado por el navegador a la carpeta de descargas.


In [ ]:
from google.colab import files
# Drop the 'text' column as requested
if 'text' in df_new.columns:
    df_new = df_new.drop(columns=['text'])
if 'groups_list' in df_new.columns:
    df_new = df_new.drop(columns=['groups_list'])
# Display the first 10 rows of the DataFrame with the new 'group_predicted' column
print("\nPrimeras 10 filas del DataFrame con la columna 'group_predicted':")
display(df_new.head(10))

# (Optional) Save the updated DataFrame to a new CSV file
df_new.to_csv('predictions_values.csv', index=False)
print("\nDataFrame actualizado guardado en 'predictions_values.csv'")
try:
  files.download('predictions_values.csv')
  print("\nVer archivo en la carpeta de 'Descargas' de tu computador")
except:
  print("\nNo se pudo descargar el archivo al computador")

# Resumen:

## Hallazgos Clave del Análisis de Datos

* El sistema cargó exitosamente un archivo CSV proporcionado por el usuario en un DataFrame de pandas, verificando su estructura y contenido.
* Se creó una nueva columna 'text' combinando las columnas 'title' y 'abstract' de los datos de entrada, para ayudar a la vectorización.
* El `TfidfVectorizer` previamente entrenado transformó exitosamente la columna 'text' en una matriz TF-IDF con dimensiones (3565, 5000).
* El modelo `LinearSVC` entrenado generó predicciones multi-etiqueta para las 3565 entradas, resultando en un array de predicción de forma (3565, 4).
* Las etiquetas binarias predichas se convirtieron de nuevo a nombres de grupo y se añadieron como una nueva columna 'group_predicted' al DataFrame.
* Cuando el archivo de entrada contenía una columna 'group' original, se calcularon y mostraron métricas de evaluación, incluyendo un informe de clasificación y matrices de confusión para cada clase, mostrando el rendimiento del modelo en los nuevos datos.
* Se mostraron las primeras 10 filas del DataFrame actualizado, incluyendo la columna 'group_predicted'.

### Insights

* El proceso integró exitosamente un modelo y vectorizador pre-entrenados para hacer predicciones en datos nuevos y no vistos proporcionados por el usuario, cumpliendo el requisito principal de la tarea.
* Se evaluaron otros modelos como redes neuronales con tensorflow.Keras, sin embargo, no se tuvieron resultados satisfactorios en las pruebas realizadas pre-eliminarmente y los tiempos de entrenamiento y respuesta de maquina eran mucho mayores a los observados con sklearn.
* Se lograron valores para el F1-score del 0.98 con el data set original, solo el subgrupo oncological tiene un menor valor, debido a que fue el grupo con menor cantidad de datos en el DataSet de entrenamiento para el modelo, sin embargo llego al 0.98.
* Se implemeta el paso de guardar el DataFrame actualizado en un nuevo archivo CSV (`predictions_values.csv`) para proporcionar un archivo de salida fácilmente utilizable para el usuario.